# Experiment Initialization

Here, the terms of the experiment are defined, such as the location of the files in S3 (bucket and folder name), and each of the video prefixes (everything before the file extension) that need to be tracked. 

Note that these videos should be similar-ish: while we can account for differences in mean intensities between videos, particle sizes should be approximately the same, and (slightly less important) particles should be moving at about the same order of magnitude speed. In this experiment, these videos were taken in 0.4% agarose gel at 100x magnification and 100.02 fps shutter speeds with nanoparticles of about 100nm in diameter.

In [1]:
to_track = [] # This is going to be the list of all filenames that will be included in the analysis
start_knot = 75 #Must be unique number for every run on Cloudknot.

remote_folder = '02_21_20_OGD_Severity_MPT' # The folder in AWS S3 containing the files to be analyzed
bucket = 'mckenna.data' # The bucket in AWS S3 where the remote_folder is contained
vids = 5 # this is the number of vids that were taken per condition (usually corresponding to different locations)
conditions = ['NT', 'OGD_3h']
NT_slices = 2
OGD_3h_slices = 3
NT_regions = ['cortex', 'hippocampus', 'striatum']
OGD_3h_regions = ['cortex', 'striatum']
for cond in conditions:
    if cond == 'NT':
        for slic in range(1,NT_slices+1):
            if slic == 1:
                for reg in NT_regions:
                    if reg == 'cortex':
                        for num in range(1,11):
                            to_track.append('{}_slice_{}_{}_vid_{}'.format(cond, slic, reg, num))
                    elif reg == 'hippocampus':
                        for num in range(1,7):
                            to_track.append('{}_slice_{}_{}_vid_{}'.format(cond, slic, reg, num))
                    else:
                        for num in range(1,vids+1):
                            to_track.append('{}_slice_{}_{}_vid_{}'.format(cond, slic, reg, num))
            else:
                for reg in NT_regions:
                    for num in range(1,vids+1):
                        to_track.append('{}_slice_{}_{}_vid_{}'.format(cond, slic, reg, num))
    else:
        for slic in range(1,OGD_3h_slices+1):
            for reg in OGD_3h_regions:
                for num in range(1,vids+1):
                    to_track.append('{}_slice_{}_{}_vid_{}'.format(cond, slic, reg, num))

In [2]:
to_track

['NT_slice_1_cortex_vid_1',
 'NT_slice_1_cortex_vid_2',
 'NT_slice_1_cortex_vid_3',
 'NT_slice_1_cortex_vid_4',
 'NT_slice_1_cortex_vid_5',
 'NT_slice_1_cortex_vid_6',
 'NT_slice_1_cortex_vid_7',
 'NT_slice_1_cortex_vid_8',
 'NT_slice_1_cortex_vid_9',
 'NT_slice_1_cortex_vid_10',
 'NT_slice_1_hippocampus_vid_1',
 'NT_slice_1_hippocampus_vid_2',
 'NT_slice_1_hippocampus_vid_3',
 'NT_slice_1_hippocampus_vid_4',
 'NT_slice_1_hippocampus_vid_5',
 'NT_slice_1_hippocampus_vid_6',
 'NT_slice_1_striatum_vid_1',
 'NT_slice_1_striatum_vid_2',
 'NT_slice_1_striatum_vid_3',
 'NT_slice_1_striatum_vid_4',
 'NT_slice_1_striatum_vid_5',
 'NT_slice_2_cortex_vid_1',
 'NT_slice_2_cortex_vid_2',
 'NT_slice_2_cortex_vid_3',
 'NT_slice_2_cortex_vid_4',
 'NT_slice_2_cortex_vid_5',
 'NT_slice_2_hippocampus_vid_1',
 'NT_slice_2_hippocampus_vid_2',
 'NT_slice_2_hippocampus_vid_3',
 'NT_slice_2_hippocampus_vid_4',
 'NT_slice_2_hippocampus_vid_5',
 'NT_slice_2_striatum_vid_1',
 'NT_slice_2_striatum_vid_2',
 'NT_s

The videos used with this analysis are fairly large (2048 x 2048 pixels and 651 frames), and in cases like this, the tracking algorithm can quickly eat up RAM. In this case, we chose to crop the videos to 512 x 512 images such that we can run our jobs on smaller EC2 instances with 16GB of RAM. 

Note that larger jobs can be made with user-defined functions such that splitting isn't necessary-- or perhaps an intermediate amount of memory that contains splitting, tracking, and msd calculation functions all performed on a single EC2 instance.

The compiled functions in the knotlets module require access to buckets on AWS. In this case, we will be using a publicly (read-only) bucket. If users want to run this notebook on their own, will have to transfer files from nancelab.publicfiles to their own bucket, as it requires writing to S3 buckets.

In [9]:
import diff_classifier.knotlets as kn

In [ ]:
# This cell uses the function kn.split() to split all of the videos contained in 'to_track' into 16 smaller videos on which the actual tracking will be performed
for prefix in to_track:
    kn.split(prefix, remote_folder=remote_folder, bucket=bucket)

## Tracking predictor

Tracking normally requires user input in the form of tracking parameters e.g. particle radius, linking max distance, max frame gap etc. When large datasets aren't required, each video can be manageably manually tracked using the TrackMate GUI. However, when datasets get large e.g. >20 videos, this can become extremely arduous. For videos that are fairly similar, you can get away with using similar tracking parameters across all videos. However, one parameter that is a little more noisy that the others is the quality filter value. Quality is a numerical value that approximate how likely a particle is to be "real." 

In this case, I built a predictor that estimates the quality filter value based on intensity distributions from the input images. Using a relatively small training dataset (5-20 videos), users can get fairly good estimates of quality filter values that can be used in parallelized tracking workflows.

Note: in the current setup, the predictor should be run in Python 3. While the code will run in Python 3, there are differences between the random number generators in Python2 and Python3 that I was not able to control for.

In [4]:
import os
import diff_classifier.imagej as ij
import boto3
import os.path as op
import diff_classifier.aws as aws
import diff_classifier.knotlets as kn
import numpy as np
from sklearn.externals import joblib

The regress_sys function should be run twice. When have_output is set to False, it generates a list of files that the user should manually track using Trackmate. Once the quality filter values are found, they can be used as input (y) to generate a regress object that can predict quality filter values for additional videos. Once y is assigned, set have_output to True and re-run the cell.

In [ ]:
tnum=30 #number of training datasets
pref = []
for num in to_track:                    
    for row in range(0, 4):
        for col in range(0, 4):
            pref.append("{}_{}_{}".format(num, row, col))

y = np.array([2.67, 2.11, 4.09, 5.15, 7.75, 4.25, 5.24, 2.75, 3.34, 7.44, 2.13, 1.89, 9.37, 7.07, 3.01, 6.33, 6.42, 6.44, 2.74, 2.85, 26.36, 2.79, 3.59, 5.07, 5.42, 5.95, 6.70, 3.44, 2.27, 4.27])

# Creates regression object based of training dataset composed of input images and manually
# calculated quality cutoffs from tracking with GUI interface.
regress = ij.regress_sys(remote_folder, pref, y, tnum, randselect=True,
                         have_output=True, bucket_name=bucket)
#Read up on how regress_sys works before running.

In [5]:
print(len(to_track))

66


In [ ]:
#Pickle object
filename = 'regress.obj'
with open(filename,'wb') as fp:
    joblib.dump(regress,fp)

import boto3
s3 = boto3.client('s3')
aws.upload_s3(filename, remote_folder+'/'+filename, bucket_name=bucket)

Users should input all tracking parameters into the tparams object. Note that the quality value will be overwritten by values found using the quality predictor found above. Never change threshold, median intensity, or snr

In [23]:
tparams1 = {'radius': 6.0, 'threshold': 0.0, 'do_median_filtering': False,
           'quality': 10.0, 'xdims': (0, 511), 'ydims': (1, 511),
           'median_intensity': 300.0, 'snr': 0.0, 'linking_max_distance': 12.0,
           'gap_closing_max_distance': 15.0, 'max_frame_gap': 8.0,
           'track_duration': 10.00}

## Attempting to run a single vid at a time

In [5]:
remote_folder = '02_21_20_OGD_Severity_MPT'
bucket = 'mckenna.data'
filename = 'NT_slice_1_striatum_vid_2_0_3.tif'
aws.download_s3(remote_folder+'/'+filename, filename, bucket_name=bucket)
local_name = filename


In [6]:
op.exists('/home/ubuntu/source/mike_fork/diff_classifier/notebooks/development')

True

In [7]:
vid_path = '/home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/'
target = vid_path+local_name
out_csv = 'Traj_'+local_name+'.csv'

In [21]:
import sys
sys.platform

'linux2'

In [8]:
print(target)
print(out_csv)

/home/ubuntu/source/mike_fork/diff_classifier/notebooks/development/NT_slice_1_striatum_vid_2_0_3.tif
Traj_NT_slice_1_striatum_vid_2_0_3.tif.csv


In [9]:
ij.track(local_name, out_csv, template=None, fiji_bin=None, tparams={'radius': 6.0, 'threshold': 0.0, 
                                                                     'quality': 2.67, 'do_median_filtering': False, 
                                                                     'xdims':(0,511), 'ydims':(1,511), 'median_intensity': 300.0,
                                                                     'snr':0.0, 'linking_max_distance': 15.0, 'gap_closing_max_distance': 18.0, 
                                                                     'max_frame_gap': 8, 'track_duration':10.0})




KeyError: 'track_displacement'

## second attempt

In [10]:
import diff_classifier.knotlets as kn

In [11]:
prefix = 'NT_slice_1_striatum_vid_2_0_3'
remote_folder = '02_21_20_OGD_Severity_MPT'
s3_bucket = 'mckenna.data'
tparams1={'radius': 6.0, 'threshold': 0.0, 'quality': 2.67, 'do_median_filtering': False,
         'xdims':(0,511), 'ydims':(1,511), 'median_intensity': 300.0,
         'snr':0.0, 'linking_max_distance': 15.0, 'gap_closing_max_distance': 18.0,
         'max_frame_gap': 8, 'track_duration':10.0}

In [12]:
kn.tracking(prefix, remote_folder, bucket=s3_bucket, regress_f='regress.obj', rows=4, cols=4, ires=(512,512),
            tparams=tparams1)

KeyError: 'track_displacement'

## Cloudknot setup

Cloudknot requires the user to define a function that will be sent to multiple computers to run. In this case, the function knotlets.tracking will be used. We create a docker image that has the required installations (defined by the requirements.txt file from diff_classifier on Github, and the base Docker Image below that has Fiji pre-installed in the correct location.

Note that I modify the Docker image below such that the correct version of boto3 is installed. For some reason, versions later than 1.5.28 error out, so I specified 5.28 as the correct version. Run my_image.build below to double-check that the Docker image is successfully built prior to submitting the job to Cloudknot.

** Before you run this next cell, you have to switch the kernel from Python 3 to Python 2, by doing the following: **
 1. Kernel -> Restart and clear output
 2. Kernel -> Change Kernel -> Python 2
 3. Rerun cells required to run below cell
 
 ** One other important thing to note: 
 - If you are performing the tracking, be sure that the my_image =  line is set to ck.DockerImage(func=kn.tracking,...
 - If you are performing the MSD/feature calculation (after you've carried out the tracking), be sure that the my_image = line is se to ck.DockerImage(func=kn.assemble_msds, ...
     
     ** following the tracking, before you run assemble_msds, you need to run the cell below that redefines all_maps as all_maps2. all_maps2 doesn't include the tparams1 input, and allows the kn.assemble_msds section to run properly. It won't work with the tparams input. **
     
 Other than that, everything else should stay the same

In [6]:
import cloudknot as ck
import os.path as op

github_installs=('https://github.com/ccurtis7/diff_classifier.git')
my_image = ck.DockerImage(func=kn.tracking, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)
#my_image = ck.DockerImage(func=kn.assemble_msds, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)
docker_file = open(my_image.docker_path)
docker_string = docker_file.read()
docker_file.close()

req = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'))
req_string = req.read()
req.close()

new_req = req_string[0:req_string.find('\n')-5]+'5.28'+ req_string[req_string.find('\n'):]
req_overwrite = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'), 'w')
req_overwrite.write(new_req)
req_overwrite.close()

Following the execution of this cell, you have to check that the requirements.txt file has the first line 'boto3==1.5.28'.
    - This file can be found in source -> diff-classifier -> notebooks -> development -> most recent file

If it doesn't, you may have to change the line in the cell above that says 'new_req = reg_string[0:req_string.find('\n')-4]+'5.28'+ reg_string[reg_string.find('\n'):]'

In [7]:
my_image.build("ChABC_slice_2", image_name="test_image")

In [ ]:
to_track

The object all_maps is an iterable containing all the inputs sent to Cloudknot. This is useful, because if the user needs to modify some of the tracking parameters for a single video, this can be done prior to submission to Cloudknot.

In [8]:
names = []
all_maps = []
for prefix in to_track[0:10]:    
    for i in range(0, 4):
        for j in range(0, 4):
            names.append('{}_{}_{}'.format(prefix, i, j))
            all_maps.append(('{}_{}_{}'.format(prefix, i, j), remote_folder, bucket, 'regress.obj', 4, 4, (512, 512), tparams1))

In [18]:
start_knot = 16

The Cloudknot knot object sets up the compute environment which will run the code. Note that the name must be unique. Every time you submit a new knot, you should change the name. I do this with the variable start_knot, which I vary for each run.

If larger jobs are anticipated, users can adjust both RAM and storage with the memory and image_id variables. Memory specifies the amount of RAM to be used. Users can build a customized AMI with as much space as they need, and enter the ID into image_ID. Read the Cloudknot documentation for more details.

In [19]:
knot = ck.Knot(name='{}_d{}'.format('mike', start_knot),
               docker_image = my_image,
               memory = 16000,
               resource_type = "SPOT",
               bid_percentage = 100,
               #image_id = 'ami-0e00afdf500081a0d', #May need to change this line
               pars_policies=('AmazonS3FullAccess',))

result_futures = knot.map(all_maps, starmap=True)

RetryError: RetryError[<Future at 0x7f2b5cc74610 state=finished raised NoSuchEntityException>]

In [ ]:
knot.clobber()

You can track the progression of your run using the AWS Batch service online -- make sure you are looking at the right US Region.

After the run, you might have some that fail. This usually happens when the computers get claimed by someone paying more money, and your job gets booted from the aws computers. Because of this, you will need to start a knw cloudknot knot and rerun those vids. The set up for that is shown below.

Remember to clobber your knot!

In [ ]:
ck.get_region()

This creates a new all_maps2 array for any of the videos that failed to get analyzed the first time through. Double check that it worked well by printing the length of it immediately afterwards.

In [ ]:
missing = []
all_maps2 = []
import boto3
import botocore

s3 = boto3.resource('s3')


for name in names:
    try:
        s3.Object(bucket, '{}/Traj_{}.csv'.format(remote_folder, name)).load()
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            missing.append(name)
            all_maps2.append((name, remote_folder, bucket, 'regress.obj',
                             4, 4, (512, 512), tparams1))
        else:
            print('Something else has gone wrong')

In [ ]:
print(len(all_maps2))

Make sure you change the name of your knot, either by changing the 'mike1' part or start_knot value.

In [ ]:
all_maps2 = []
for prefix in to_track:
    all_maps2.append((prefix, remote_folder, bucket, (512, 512), 651, 4, 4))

Users can monitor the progress of their job in the Batch interface. Once the code is complete, users should clobber their knot to make sure that all AWS resources are removed.

In [ ]:
to_track

## Downstream analysis and visualization

The knotlet.assemble_msds function (which can also potentially be submitted to Cloudknot as well for large jobs) calculates the mean squared displacements and trajectory features from the raw trajectory csv files found from the Cloudknot submission. It accesses them from the S3 bucket to which they were saved.

In [ ]:
for prefix in to_track:
    kn.assemble_msds(prefix, remote_folder, bucket='mckenna.data')
    print('Successfully output msds for {}'.format(prefix))

In [ ]:
for prefix in to_track[5:7]:
    kn.assemble_msds(prefix, remote_folder, bucket='ccurtis.data')
    print('Successfully output msds for {}'.format(prefix))

Diff_classifier includes some useful imaging tools as well, including checking trajectories, plotting heatmaps of trajectory features, distributions of diffusion coefficients, and MSD plots.

In [ ]:
import diff_classifier.heatmaps as hm
import diff_classifier.aws as aws
import os
import os.path as op

In [ ]:
to_track

In [ ]:
for vids in to_track[45:90]:
    prefix = vids
    msds = 'msd_{}.csv'.format(prefix)
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3('{}/{}'.format(remote_folder, msds), msds, bucket_name=bucket)
    aws.download_s3('{}/{}'.format(remote_folder, feat), feat, bucket_name=bucket)
    hm.plot_trajectories(prefix, remote_folder=remote_folder, upload=True, figsize=(8, 8), bucket = bucket)
    print('Successfully uploaded trajectory plot for {}'.format(prefix))
    geomean, geoSEM = hm.plot_individual_msds(prefix, x_range=4, y_range=0.5, umppx=0.07, fps=100, upload=True, remote_folder=remote_folder, bucket = bucket)
    aws.upload_s3('./geomean_{}.csv'.format(prefix), remote_folder+'/geomean_{}.csv'.format(prefix), bucket_name = bucket)
    aws.upload_s3('./geoSEM_{}.csv'.format(prefix), remote_folder+'/geoSEM_{}.csv'.format(prefix), bucket_name = bucket)
    aws.upload_s3('./msds_{}.png'.format(prefix), remote_folder+'/msds_{}.png'.format(prefix), bucket_name = bucket)
    os.remove('features_{}.csv'.format(prefix))
    os.remove('geoSEM_{}.csv'.format(prefix))
    os.remove('msd_{}.csv'.format(prefix))
    os.remove('geomean_{}.csv'.format(prefix))
    os.remove('msds_{}.png'.format(prefix))
    print('Successfully uploaded csv files for {}'.format(prefix))
    
    

In [ ]:
geomean, geoSEM = hm.plot_individual_msds(prefix, x_range=4, y_range=0.5, umppx=0.07, fps=100, upload=True, remote_folder=remote_folder, bucket = bucket)
aws.upload_s3('./geomean_{}.csv'.format(prefix), remote_folder+'/geomean_{}.csv'.format(prefix), bucket_name = bucket)
aws.upload_s3('./geoSEM_{}.csv'.format(prefix), remote_folder+'/geoSEM_{}.csv'.format(prefix), bucket_name = bucket)
aws.upload_s3('./msds_{}.png'.format(prefix), remote_folder+'/msds_{}.png'.format(prefix), bucket_name = bucket)

In [ ]:
hm.plot_heatmap(prefix, upload=False)

In [ ]:
hm.plot_particles_in_frame(prefix, y_range=500, upload=False)

In [ ]:
prefix = to_track[0]

msds = 'msd_{}.csv'.format(prefix)
feat = 'features_{}.csv'.format(prefix)
aws.download_s3('{}/{}'.format(remote_folder, msds), msds, bucket_name=bucket)
aws.download_s3('{}/{}'.format(remote_folder, feat), feat, bucket_name=bucket)

In [ ]:
hm.plot_trajectories(prefix, remote_folder=remote_folder, upload=True, figsize=(8, 8), bucket = bucket)

In [ ]:
geomean, geoSEM = hm.plot_individual_msds(prefix, x_range=4, y_range=0.5, umppx=0.07, fps=100, upload=True, remote_folder=remote_folder, bucket = bucket)
aws.upload_s3('./geomean_{}.csv'.format(prefix), remote_folder+'/geomean_{}.csv'.format(prefix), bucket_name = bucket)
aws.upload_s3('./geoSEM_{}.csv'.format(prefix), remote_folder+'/geoSEM_{}.csv'.format(prefix), bucket_name = bucket)
aws.upload_s3('./msds_{}.png'.format(prefix), remote_folder+'/msds_{}.png'.format(prefix), bucket_name = bucket)

## Converting msd files

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math
import os
import os.path as op
from os import listdir

In [ ]:
to_track = [] # This is going to be the list of all filenames that will be included in the analysis
start_knot = 80 #Must be unique number for every run on Cloudknot.

remote_folder = '08_06_19_MPT_age_dependence' # The folder in AWS S3 containing the files to be analyzed
bucket = 'mckenna.data' # The bucket in AWS S3 where the remote_folder is contained
NP_sizes = ['40','100']
vids = 5 # this is the number of vids that were taken per condition (usually corresponding to different locations)
ages = ['P14', 'P21', 'P28']
slices = 3
for size in NP_sizes:
    for age in ages:
        for slic in range(1, slices+1):
            for num in range(1, vids+1):
                to_track.append('{}_{}nm_s{}_v{}'.format(age, size, slic, num))

In [ ]:
to_track

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math

for prefix in to_track[45:90]:
    filename = 'geomean_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+filename, filename, bucket_name=bucket)
    local_name = filename
    merged = pd.read_csv(local_name)
    merged.columns = ['log']
    merged['exp'] = 0
    for rows in range(0,len(merged)):
        log_value = merged['log'].iloc[rows]
        exp_value = math.exp(log_value)
        merged.loc[rows,'exp'] = exp_value
    merged.to_csv('adj_'+filename, mode='w', index = False)
    aws.upload_s3('./adj_'+filename, remote_folder+'/adj_'+filename, bucket_name = bucket)
    os.remove(filename)
    os.remove('adj_'+filename)

In [ ]:
pref = 'adj_geomean_'

mol_wts = ['high', 'low', 'med']
num_vids = 5

for MW in mol_wts:
    avg_df = pd.DataFrame()
    for vid in range(1, num_vids+1):
        exp = pd.read_csv(pref+'{}MW_40nm_vid_{}.csv'.format(MW, str(vid)))
        avg_df = pd.concat([avg_df, exp['exp']], axis=1)
    avg_df = avg_df.mean(axis = 1)
    avg_df.to_csv('avg_'+pref+'{}MW_40nm.csv'.format(MW), mode='w', index = False)
    aws.upload_s3('./avg_'+pref+'{}MW_40nm.csv'.format(MW), remote_folder+'/avg_'+pref+'{}MW_40nm.csv'.format(MW), bucket_name=bucket)
    

## Generating Histograms

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math
import os
import os.path as op
from os import listdir

In [ ]:
to_track

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math

high_track = to_track[0:5]
low_track = to_track[5:10]
med_track = to_track[10:15]

low_MW = pd.DataFrame()
med_MW = pd.DataFrame()
high_MW = pd.DataFrame()

um_px = 0.07
fps = 33

for prefix in low_track:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, 'trial_2_'+feat, bucket_name=bucket)
    merged = pd.read_csv('trial_2_'+feat)
    low_MW = pd.concat([low_MW, merged['Deff1']*um_px*um_px*fps/10], axis=0)
    
for prefix in med_track:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, 'trial_2_'+feat, bucket_name=bucket)
    merged = pd.read_csv('trial_2_'+feat)
    med_MW = pd.concat([med_MW, merged['Deff1']*um_px*um_px*fps/10], axis=0)
    
for prefix in high_track:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, 'trial_2_'+feat, bucket_name=bucket)
    merged = pd.read_csv('trial_2_'+feat)
    high_MW = pd.concat([high_MW, merged['Deff1']*um_px*um_px*fps/10], axis=0)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma

low_no_nan = low_MW.replace(0,np.nan)
med_no_nan = med_MW.replace(0,np.nan)
high_no_nan = high_MW.replace(0,np.nan)


log_Deff_low = np.log(low_no_nan[0].dropna())
#print(log_Deff_low)
log_Deff_med = np.log(med_no_nan[0].dropna())
#print(log_Deff_low)
log_Deff_high = np.log(high_no_nan[0].dropna())

test_bins = np.linspace(-12, 0, 76)

low_hist, low_bins = np.histogram(log_Deff_low, bins=test_bins)
med_hist, med_bins = np.histogram(log_Deff_med, bins=test_bins)
high_hist, high_bins = np.histogram(log_Deff_high, bins=test_bins)

low_avg = np.mean(log_Deff_low)
med_avg = np.mean(log_Deff_med)
high_avg = np.mean(log_Deff_high)

plt.rc('axes', linewidth=2)
low_plot, med_plot, high_plot = low_hist, med_hist, high_hist
bins = test_bins
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:])/2

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, high_plot, color='royalblue', align='center', width=width, alpha=0.6, label='high MW')
plt.axvline(high_avg, color='royalblue', linestyle='--', linewidth=3)
plt.ylim((0,800))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, med_plot, color='darkgreen', align='center', width=width, alpha=0.7, label='medium MW')
plt.axvline(med_avg, color='darkgreen', linestyle='--', linewidth=3)
plt.ylim((0,400))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, low_plot, color='firebrick', align='center', width=width, alpha=1, label='low MW')
plt.axvline(low_avg, color='firebrick', linestyle='--', linewidth=3)
plt.ylim((0,100))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.ylim((0,600))
plt.legend(fontsize='large', loc=2)
plt.show()

In [ ]:
print(low_avg, med_avg, high_avg)
print(np.exp(low_avg), np.exp(med_avg), np.exp(high_avg))

## Combined HA Solution Histograms

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math
import os
import os.path as op
from os import listdir
from scipy import stats

In [ ]:
to_track = [] # This is going to be the list of all filenames that will be included in the analysis
start_knot = 80 #Must be unique number for every run on Cloudknot.

remote_folder = '08_06_19_MPT_age_dependence' # The folder in AWS S3 containing the files to be analyzed
bucket = 'mckenna.data' # The bucket in AWS S3 where the remote_folder is contained
NP_sizes = ['40','100']
vids = 5 # this is the number of vids that were taken per condition (usually corresponding to different locations)
ages = ['P14', 'P21', 'P28']
slices = 3
for size in NP_sizes:
    for age in ages:
        for slic in range(1, slices+1):
            for num in range(1, vids+1):
                to_track.append('{}_{}nm_s{}_v{}'.format(age, size, slic, num))

In [ ]:
print(len(to_track))
to_track

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math

P14_track_40 = to_track[0:15]
P21_track_40 = to_track[15:30]
P28_track_40 = to_track[30:45]
P14_track_100 = to_track[45:60]
P21_track_100 = to_track[60:75]
P28_track_100 = to_track[75:90]

P14_40 = pd.DataFrame()
P21_40 = pd.DataFrame()
P28_40 = pd.DataFrame()
P14_100 = pd.DataFrame()
P21_100 = pd.DataFrame()
P28_100 = pd.DataFrame()

um_px = 0.07
fps_40 = 33
fps_100 = 100

for prefix in P14_track_40:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    P14_40 = pd.concat([P14_40, merged['Deff1']*um_px*um_px*fps_40/10], axis=0)
    os.remove(feat)
P14_40.to_csv('P14_40nm.csv', mode='w', index = False)
    
for prefix in P21_track_40:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    P21_40 = pd.concat([P21_40, merged['Deff1']*um_px*um_px*fps_40/10], axis=0)
    os.remove(feat)
P21_40.to_csv('P21_40nm.csv', mode='w', index = False)
    
for prefix in P28_track_40:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    P28_40 = pd.concat([P28_40, merged['Deff1']*um_px*um_px*fps_40/10], axis=0)
    os.remove(feat)
P28_40.to_csv('P28_40nm.csv', mode='w', index = False)    
    
for prefix in P14_track_100:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    P14_100 = pd.concat([P14_100, merged['Deff1']*um_px*um_px*fps_100/10], axis=0)
    os.remove(feat)
P14_100.to_csv('P14_100nm.csv', mode='w', index = False)
    
for prefix in P21_track_100:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    P21_100 = pd.concat([P21_100, merged['Deff1']*um_px*um_px*fps_100/10], axis=0)
    os.remove(feat)
P21_100.to_csv('P21_100nm.csv', mode='w', index = False)
    
for prefix in P28_track_100:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    P28_100 = pd.concat([P28_100, merged['Deff1']*um_px*um_px*fps_100/10], axis=0)
    os.remove(feat)
P28_100.to_csv('P28_100nm.csv', mode='w', index = False)

In [ ]:
to_track = [] # This is going to be the list of all filenames that will be included in the analysis
start_knot = 80 #Must be unique number for every run on Cloudknot.

remote_folder = '07_10_19_MPT_HA_solutions_trial2' # The folder in AWS S3 containing the files to be analyzed
bucket = 'mckenna.data' # The bucket in AWS S3 where the remote_folder is contained
NP_sizes = ['40', '100']
vids = 5 # this is the number of vids that were taken per condition (usually corresponding to different locations)
mol_weights = ['high', 'low', 'med']

for MW in mol_weights:
    for size in NP_sizes:
        for num in range(1, vids+1):
            to_track.append('{}MW_{}nm_vid_{}'.format(MW, size, num))
            


In [ ]:
print(len(to_track))
to_track

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math

high_track_40 = to_track[0:5]
low_track_40 = to_track[10:15]
med_track_40 = to_track[20:25]
high_track_100 = to_track[5:10]
low_track_100 = to_track[15:20]
med_track_100 = to_track[25:30]


low_MW_40 = pd.DataFrame()
med_MW_40 = pd.DataFrame()
high_MW_40 = pd.DataFrame()
low_MW_100 = pd.DataFrame()
med_MW_100 = pd.DataFrame()
high_MW_100 = pd.DataFrame()

um_px = 0.07
fps_40 = 33
fps_100 = 100

for prefix in low_track_40:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    low_MW_40 = pd.concat([low_MW_40, merged['Deff1']*um_px*um_px*fps_40/10], axis=0)
    os.remove(feat)
low_MW_40.to_csv('40nm_lowMW_trial_2.csv', mode='w', index = False)
    
for prefix in med_track_40:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    med_MW_40 = pd.concat([med_MW_40, merged['Deff1']*um_px*um_px*fps_40/10], axis=0)
    os.remove(feat)
med_MW_40.to_csv('40nm_medMW_trial_2.csv', mode='w', index = False)
    
for prefix in high_track_40:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    high_MW_40 = pd.concat([high_MW_40, merged['Deff1']*um_px*um_px*fps_40/10], axis=0)
    os.remove(feat)
high_MW_40.to_csv('40nm_highMW_trial_2.csv', mode='w', index = False)    
    
for prefix in low_track_100:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    low_MW_100 = pd.concat([low_MW_100, merged['Deff1']*um_px*um_px*fps_100/10], axis=0)
    os.remove(feat)
low_MW_100.to_csv('100nm_lowMW_trial_2.csv', mode='w', index = False)
    
for prefix in med_track_100:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    med_MW_100 = pd.concat([med_MW_100, merged['Deff1']*um_px*um_px*fps_100/10], axis=0)
    os.remove(feat)
med_MW_100.to_csv('100nm_medMW_trial_2.csv', mode='w', index = False)
    
for prefix in high_track_100:
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    high_MW_100 = pd.concat([high_MW_100, merged['Deff1']*um_px*um_px*fps_100/10], axis=0)
    os.remove(feat)
high_MW_100.to_csv('100nm_highMW_trial_2.csv', mode='w', index = False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma

P14_40nm = pd.read_csv('P14_40nm.csv')
P21_40nm = pd.read_csv('P21_40nm.csv')
P28_40nm = pd.read_csv('P28_40nm.csv')
NT_brain_2 = pd.read_csv('NT_brain_2_hist.csv')

P35_40nm = pd.DataFrame()
P35_40nm = pd.concat([NT_brain_2], axis=0)

P14_40_no_nan = P14_40nm.replace(0,np.nan)
P21_40_no_nan = P21_40nm.replace(0,np.nan)
P28_40_no_nan = P28_40nm.replace(0,np.nan)
P35_40_no_nan = P35_40nm.replace(0,np.nan)

log_Deff_P14_40 = np.log(P14_40_no_nan.dropna())
log_Deff_P21_40 = np.log(P21_40_no_nan.dropna())
log_Deff_P28_40 = np.log(P28_40_no_nan.dropna())
log_Deff_P35_40 = np.log(P35_40_no_nan.dropna())

test_bins = np.linspace(-10, 2, 76)

P14_hist_40, P14_bins_40 = np.histogram(log_Deff_P14_40, bins=test_bins)
P21_hist_40, P21_bins_40 = np.histogram(log_Deff_P21_40, bins=test_bins)
P28_hist_40, P28_bins_40 = np.histogram(log_Deff_P28_40, bins=test_bins)
P35_hist_40, P35_bins_40 = np.histogram(log_Deff_P35_40, bins=test_bins)

P14_avg_40 = np.mean(log_Deff_P14_40)[0]
P21_avg_40 = np.mean(log_Deff_P21_40)[0]
P28_avg_40 = np.mean(log_Deff_P28_40)[0]
P35_avg_40 = np.mean(log_Deff_P35_40)[0]

plt.rc('axes', linewidth=2)
P14_plot_40, P21_plot_40, P28_plot_40, P35_plot_40 = P14_hist_40, P21_hist_40, P28_hist_40, P35_hist_40
bins = test_bins
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:])/2

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, P14_plot_40, color='grey', align='center', width=width, alpha=0.8, label='P14')
plt.axvline(P14_avg_40, color='grey', linestyle='--', linewidth=3)
plt.ylim((0,800))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, P21_plot_40, color='goldenrod', align='center', width=width, alpha=0.7, label='P21')
plt.axvline(P21_avg_40, color='goldenrod', linestyle='--', linewidth=3)
plt.ylim((0,400))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, P28_plot_40, color='royalblue', align='center', width=width, alpha=0.7, label='P28')
plt.axvline(P28_avg_40, color='royalblue', linestyle='--', linewidth=3)
plt.ylim((0,100))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, P35_plot_40, color='purple', align='center', width=width, alpha=0.7, label='P35')
plt.axvline(P35_avg_40, color='purple', linestyle='--', linewidth=3)
plt.ylim((0,100))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.ylim((0,1400))
plt.legend(fontsize='x-large', loc=2)
plt.show()

## Statistics

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math
import os
import os.path as op
from os import listdir
import numpy as np

In [ ]:
to_track[0:45]

In [ ]:
remote_folder

In [ ]:

um_px = 0.07
fps = 33

for prefix in to_track[0:45]:
    temp = pd.DataFrame()
    feat = 'features_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+feat, feat, bucket_name=bucket)
    merged = pd.read_csv(feat)
    temp = pd.concat([temp, merged['Deff1']*um_px*um_px/(1/fps*10)], axis=0)
    temp_no_nan = temp.replace(0,np.nan)
    temp_upload = temp_no_nan.dropna()
    temp_upload.to_csv('{}_stats.csv'.format(prefix), mode='w', index = False)
    aws.upload_s3('{}_stats.csv'.format(prefix), '07_16_19_ECM_Breakdown/{}_stats.csv'.format(prefix), bucket_name='mckenna.data')
    os.remove(feat)
    os.remove('{}_stats.csv'.format(prefix))

In [ ]:
print(P14_avg_40, P21_avg_40, P28_avg_40, P35_avg_40)
print(np.exp(P14_avg_40), np.exp(P21_avg_40), np.exp(P28_avg_40), np.exp(P35_avg_40))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma

P14_40nm = pd.read_csv('P14_40nm.csv')
P21_40nm = pd.read_csv('P21_40nm.csv')
P28_40nm = pd.read_csv('P28_40nm.csv')
NT_brain_2 = pd.read_csv('NT_brain_2_hist.csv')

P35_40nm = pd.DataFrame()
P35_40nm = pd.concat([NT_brain_2], axis=0)

P14_40_no_nan = P14_40nm.replace(0,np.nan)
P21_40_no_nan = P21_40nm.replace(0,np.nan)
P28_40_no_nan = P28_40nm.replace(0,np.nan)
P35_40_no_nan = P35_40nm.replace(0,np.nan)

log_Deff_P14_40 = np.log(P14_40_no_nan.dropna())
log_Deff_P21_40 = np.log(P21_40_no_nan.dropna())
log_Deff_P28_40 = np.log(P28_40_no_nan.dropna())
log_Deff_P35_40 = np.log(P35_40_no_nan.dropna())

test_bins = np.linspace(-10, 2, 76)

P14_hist_40, P14_bins_40 = np.histogram(log_Deff_P14_40, bins=test_bins)
P21_hist_40, P21_bins_40 = np.histogram(log_Deff_P21_40, bins=test_bins)
P28_hist_40, P28_bins_40 = np.histogram(log_Deff_P28_40, bins=test_bins)
P35_hist_40, P35_bins_40 = np.histogram(log_Deff_P35_40, bins=test_bins)

P14_avg_40 = np.mean(log_Deff_P14_40)[0]
P21_avg_40 = np.mean(log_Deff_P21_40)[0]
P28_avg_40 = np.mean(log_Deff_P28_40)[0]
P35_avg_40 = np.mean(log_Deff_P35_40)[0]

plt.rc('axes', linewidth=2)
P14_plot_40, P21_plot_40, P28_plot_40, P35_plot_40 = P14_hist_40, P21_hist_40, P28_hist_40, P35_hist_40
bins = test_bins
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:])/2

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, P21_plot_40, color='goldenrod', align='center', width=width, alpha=0.7, label='P21')
plt.axvline(P21_avg_40, color='goldenrod', linestyle='--', linewidth=3)
plt.ylim((0,400))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.ylim((0,1400))
plt.legend(fontsize='x-large', loc=2)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma

P14_40nm = pd.read_csv('P14_40nm.csv')
P21_40nm = pd.read_csv('P21_40nm.csv')
P28_40nm = pd.read_csv('P28_40nm.csv')
NT_brain_2 = pd.read_csv('NT_brain_2_hist.csv')

P35_40nm = pd.DataFrame()
P35_40nm = pd.concat([NT_brain_2], axis=0)

P14_40_no_nan = P14_40nm.replace(0,np.nan)
P21_40_no_nan = P21_40nm.replace(0,np.nan)
P28_40_no_nan = P28_40nm.replace(0,np.nan)
P35_40_no_nan = P35_40nm.replace(0,np.nan)

log_Deff_P14_40 = np.log(P14_40_no_nan.dropna())
log_Deff_P21_40 = np.log(P21_40_no_nan.dropna())
log_Deff_P28_40 = np.log(P28_40_no_nan.dropna())
log_Deff_P35_40 = np.log(P35_40_no_nan.dropna())

test_bins = np.linspace(-10, 2, 76)

P14_hist_40, P14_bins_40 = np.histogram(log_Deff_P14_40, bins=test_bins)
P21_hist_40, P21_bins_40 = np.histogram(log_Deff_P21_40, bins=test_bins)
P28_hist_40, P28_bins_40 = np.histogram(log_Deff_P28_40, bins=test_bins)
P35_hist_40, P35_bins_40 = np.histogram(log_Deff_P35_40, bins=test_bins)

P14_avg_40 = np.mean(log_Deff_P14_40)[0]
P21_avg_40 = np.mean(log_Deff_P21_40)[0]
P28_avg_40 = np.mean(log_Deff_P28_40)[0]
P35_avg_40 = np.mean(log_Deff_P35_40)[0]

plt.rc('axes', linewidth=2)
P14_plot_40, P21_plot_40, P28_plot_40, P35_plot_40 = P14_hist_40, P21_hist_40, P28_hist_40, P35_hist_40
bins = test_bins
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:])/2

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, P35_plot_40, color='purple', align='center', width=width, alpha=0.7, label='P35')
plt.axvline(P35_avg_40, color='purple', linestyle='--', linewidth=3)
plt.ylim((0,100))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.ylim((0,1400))
plt.legend(fontsize='x-large', loc=2)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma

P14_40nm = pd.read_csv('P14_40nm.csv')
P21_40nm = pd.read_csv('P21_40nm.csv')
P28_40nm = pd.read_csv('P28_40nm.csv')
NT_brain_2 = pd.read_csv('NT_brain_2_hist.csv')

P35_40nm = pd.DataFrame()
P35_40nm = pd.concat([NT_brain_2], axis=0)

P14_40_no_nan = P14_40nm.replace(0,np.nan)
P21_40_no_nan = P21_40nm.replace(0,np.nan)
P28_40_no_nan = P28_40nm.replace(0,np.nan)
P35_40_no_nan = P35_40nm.replace(0,np.nan)

log_Deff_P14_40 = np.log(P14_40_no_nan.dropna())
log_Deff_P21_40 = np.log(P21_40_no_nan.dropna())
log_Deff_P28_40 = np.log(P28_40_no_nan.dropna())
log_Deff_P35_40 = np.log(P35_40_no_nan.dropna())

test_bins = np.linspace(-10, 2, 76)

P14_hist_40, P14_bins_40 = np.histogram(log_Deff_P14_40, bins=test_bins)
P21_hist_40, P21_bins_40 = np.histogram(log_Deff_P21_40, bins=test_bins)
P28_hist_40, P28_bins_40 = np.histogram(log_Deff_P28_40, bins=test_bins)
P35_hist_40, P35_bins_40 = np.histogram(log_Deff_P35_40, bins=test_bins)

P14_avg_40 = np.mean(log_Deff_P14_40)[0]
P21_avg_40 = np.mean(log_Deff_P21_40)[0]
P28_avg_40 = np.mean(log_Deff_P28_40)[0]
P35_avg_40 = np.mean(log_Deff_P35_40)[0]

plt.rc('axes', linewidth=2)
P14_plot_40, P21_plot_40, P28_plot_40, P35_plot_40 = P14_hist_40, P21_hist_40, P28_hist_40, P35_hist_40
bins = test_bins
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:])/2

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, P14_plot_40, color='grey', align='center', width=width, alpha=0.8, label='P14')
plt.axvline(P14_avg_40, color='grey', linestyle='--', linewidth=3)
plt.ylim((0,800))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.ylim((0,1400))
plt.legend(fontsize='x-large', loc=2)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma

P14_40nm = pd.read_csv('P14_40nm.csv')
P21_40nm = pd.read_csv('P21_40nm.csv')
P28_40nm = pd.read_csv('P28_40nm.csv')
NT_brain_2 = pd.read_csv('NT_brain_2_hist.csv')

P35_40nm = pd.DataFrame()
P35_40nm = pd.concat([NT_brain_2], axis=0)

P14_40_no_nan = P14_40nm.replace(0,np.nan)
P21_40_no_nan = P21_40nm.replace(0,np.nan)
P28_40_no_nan = P28_40nm.replace(0,np.nan)
P35_40_no_nan = P35_40nm.replace(0,np.nan)

log_Deff_P14_40 = np.log(P14_40_no_nan.dropna())
log_Deff_P21_40 = np.log(P21_40_no_nan.dropna())
log_Deff_P28_40 = np.log(P28_40_no_nan.dropna())
log_Deff_P35_40 = np.log(P35_40_no_nan.dropna())

test_bins = np.linspace(-10, 2, 76)

P14_hist_40, P14_bins_40 = np.histogram(log_Deff_P14_40, bins=test_bins)
P21_hist_40, P21_bins_40 = np.histogram(log_Deff_P21_40, bins=test_bins)
P28_hist_40, P28_bins_40 = np.histogram(log_Deff_P28_40, bins=test_bins)
P35_hist_40, P35_bins_40 = np.histogram(log_Deff_P35_40, bins=test_bins)

P14_avg_40 = np.mean(log_Deff_P14_40)[0]
P21_avg_40 = np.mean(log_Deff_P21_40)[0]
P28_avg_40 = np.mean(log_Deff_P28_40)[0]
P35_avg_40 = np.mean(log_Deff_P35_40)[0]

plt.rc('axes', linewidth=2)
P14_plot_40, P21_plot_40, P28_plot_40, P35_plot_40 = P14_hist_40, P21_hist_40, P28_hist_40, P35_hist_40
bins = test_bins
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:])/2

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, P28_plot_40, color='royalblue', align='center', width=width, alpha=0.7, label='P28')
plt.axvline(P28_avg_40, color='royalblue', linestyle='--', linewidth=3)
plt.ylim((0,100))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.ylim((0,1400))
plt.legend(fontsize='x-large', loc=2)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma

P14_100nm = pd.read_csv('P14_100nm.csv')
P21_100nm = pd.read_csv('P21_100nm.csv')
P28_100nm = pd.read_csv('P28_100nm.csv')
#NT_brain_2 = pd.read_csv('NT_brain_2_hist.csv')

#P35_40nm = pd.DataFrame()
#P35_40nm = pd.concat([NT_brain_2], axis=0)

P14_100_no_nan = P14_100nm.replace(0,np.nan)
P21_100_no_nan = P21_100nm.replace(0,np.nan)
P28_100_no_nan = P28_100nm.replace(0,np.nan)
#P35_40_no_nan = P35_40nm.replace(0,np.nan)

log_Deff_P14_100 = np.log(P14_100_no_nan.dropna())
log_Deff_P21_100 = np.log(P21_100_no_nan.dropna())
log_Deff_P28_100 = np.log(P28_100_no_nan.dropna())
#log_Deff_P35_40 = np.log(P35_40_no_nan.dropna())

test_bins = np.linspace(-5, 1, 76)

P14_hist_100, P14_bins_100 = np.histogram(log_Deff_P14_100, bins=test_bins)
P21_hist_100, P21_bins_100 = np.histogram(log_Deff_P21_100, bins=test_bins)
P28_hist_100, P28_bins_100 = np.histogram(log_Deff_P28_100, bins=test_bins)
#P35_hist_40, P35_bins_40 = np.histogram(log_Deff_P35_40, bins=test_bins)

P14_avg_100 = np.mean(log_Deff_P14_100)[0]
P21_avg_100 = np.mean(log_Deff_P21_100)[0]
P28_avg_100 = np.mean(log_Deff_P28_100)[0]
#P35_avg_40 = np.mean(log_Deff_P35_40)[0]

plt.rc('axes', linewidth=2)
P14_plot_100, P21_plot_100, P28_plot_100 = P14_hist_100, P21_hist_100, P28_hist_100
bins = test_bins
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:])/2

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, P14_plot_100, color='royalblue', align='center', width=width, alpha=0.8, label='P14')
plt.axvline(P14_avg_100, color='royalblue', linestyle='--', linewidth=3)
plt.ylim((0,800))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, P21_plot_100, color='darkgreen', align='center', width=width, alpha=0.9, label='P21')
plt.axvline(P21_avg_100, color='darkgreen', linestyle='--', linewidth=3)
plt.ylim((0,400))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.figure(1, figsize=(7,4)).tight_layout()
plt.bar(center, P28_plot_100, color='firebrick', align='center', width=width, alpha=0.7, label='P28')
plt.axvline(P28_avg_100, color='firebrick', linestyle='--', linewidth=3)
plt.ylim((0,100))
plt.xlabel('log $D_{eff}$', fontsize=20)
plt.ylabel('Trajectory Count', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.ylim((0,2000))
plt.legend()
plt.show()

In [ ]:
print(P14_avg_100, P21_avg_100, P28_avg_100)
print(np.exp(P14_avg_100), np.exp(P21_avg_100), np.exp(P28_avg_100))

## MSD plots

In [ ]:
import pandas as pd
import diff_classifier.aws as aws
import math
import numpy as np

In [ ]:
for prefix in to_track[0:45]:
    filename = 'adj_geomean_{}.csv'.format(prefix)
    aws.download_s3(remote_folder+'/'+filename, filename, bucket_name=bucket)

In [ ]:
for prefix in to_track[0:45]:
    filename = 'adj_geomean_{}.csv'.format(prefix)
    os.remove(filename)

In [ ]:
# This is importing P35 dataset

to_track_P35 = [] # This is going to be the list of all filenames that will be included in the analysis
start_knot = 80 #Must be unique number for every run on Cloudknot.

remote_folder_P35 = '07_16_19_MPT_ECM_breakdown' # The folder in AWS S3 containing the files to be analyzed
bucket = 'mckenna.data' # The bucket in AWS S3 where the remote_folder is contained

treatments = ['NT']
brains = 4
slices = 3
vids = 5

for treat in treatments:
    for brain in range(1, brains+1):
        for slic in range(1, slices+1):
            for num in range(1, vids+1):
                to_track_P35.append('{}_brain_{}_slice_{}_vid_{}'.format(treat, brain, slic, num))

In [ ]:
to_track_P35

In [ ]:
import matplotlib.pyplot as plt

P14_MSDs = pd.DataFrame()
P21_MSDs = pd.DataFrame()
P28_MSDs = pd.DataFrame()
P35_MSDs = pd.DataFrame()

P14_plot_array = []
P21_plot_array = []
P28_plot_array = []
P35_plot_array = []

P14_MSDs['time'] = pd.Series(np.linspace(1/33, 1/33*650, 650))
P21_MSDs['time'] = pd.Series(np.linspace(1/33, 1/33*650, 650))
P28_MSDs['time'] = pd.Series(np.linspace(1/33, 1/33*650, 650))
P35_MSDs['time'] = pd.Series(np.linspace(1/33, 1/33*650, 650))

for prefix in to_track[0:15]:
    filename = 'adj_geomean_{}.csv'.format(prefix)
    P14_MSDs = pd.concat([P14_MSDs, pd.read_csv(filename)['exp']], axis = 1)
    P14_MSDs.rename(columns={"exp": prefix}, inplace=True)
    P14_plot_array.append(prefix)

P14_MSDs['average'] = P14_MSDs.mean(numeric_only=True, axis=1)

for prefix in to_track[15:30]:
    filename = 'adj_geomean_{}.csv'.format(prefix)
    P21_MSDs = pd.concat([P21_MSDs, pd.read_csv(filename)['exp']], axis = 1)
    P21_MSDs.rename(columns={"exp": prefix}, inplace=True)
    P21_plot_array.append(prefix)
    
P21_MSDs['average'] = P21_MSDs.mean(numeric_only=True, axis=1)

for prefix in to_track[30:45]:
    filename = 'adj_geomean_{}.csv'.format(prefix)
    P28_MSDs = pd.concat([P28_MSDs, pd.read_csv(filename)['exp']], axis = 1)
    P28_MSDs.rename(columns={"exp": prefix}, inplace=True)
    P28_plot_array.append(prefix)

P28_MSDs['average'] = P28_MSDs.mean(numeric_only=True, axis=1)

for prefix in to_track_P35:
    filename = 'adj_geomean_{}.csv'.format(prefix)
    P35_MSDs = pd.concat([P35_MSDs, pd.read_csv(filename)['exp']], axis = 1)
    P35_MSDs.rename(columns={"exp": prefix}, inplace=True)
    P35_plot_array.append(prefix)

P35_MSDs['average'] = P35_MSDs.mean(numeric_only=True, axis=1)

plt.figure(1).tight_layout()
ax = plt.gca()
P14_MSDs.plot(x='time', y=P14_plot_array, kind='line', ax=ax, xlim=(0,1.6), ylim=(0.01,100), logy=True, logx=True, color='gray', alpha=0.1, legend=False)
P14_MSDs.plot(x='time', y='average', kind='line', ax=ax, xlim=(0,1.6), ylim=(0.01,100), logy=True, logx=True, linewidth=3.0, color='gray', alpha=1, legend=False)
ax.set_ylabel('<MSD> (\u03BCm$^2$)', fontsize=18)
ax.set_xlabel('lag time (s)', fontsize=18)
ax.tick_params(labelsize=12)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)

plt.figure(2).tight_layout()
ax2 = plt.gca()
P21_MSDs.plot(x='time', y=P21_plot_array, kind='line', ax=ax2, xlim=(0,1.6), ylim=(0.01,100), logy=True, logx=True, color='goldenrod', alpha=0.1, legend=False)
P21_MSDs.plot(x='time', y='average', kind='line', ax=ax2, xlim=(0,1.6), ylim=(0.01,100), logy=True, logx=True, linewidth=3.0, color='goldenrod', alpha=1, legend=False)
ax2.set_ylabel('<MSD> (\u03BCm$^2$)', fontsize=18)
ax2.set_xlabel('lag time (s)', fontsize=18)
ax2.tick_params(labelsize=12)
for axis in ['top','bottom','left','right']:
    ax2.spines[axis].set_linewidth(2)

plt.figure(3).tight_layout()
ax3 = plt.gca()
P28_MSDs.plot(x='time', y=P28_plot_array, kind='line', ax=ax3, xlim=(0,1.6), logy=True, ylim=(0.01,100), logx=True, color='royalblue', alpha=0.1, legend=False)
P28_MSDs.plot(x='time', y='average', kind='line', ax=ax3, xlim=(0,1.6), ylim=(0.01,100), logy=True, logx=True, linewidth=3.0, color='royalblue', alpha=1, legend=False)
ax3.set_ylabel('<MSD> (\u03BCm$^2$)', fontsize=18)
ax3.set_xlabel('lag time (s)', fontsize=18)
ax3.tick_params(labelsize=12)
for axis in ['top','bottom','left','right']:
    ax3.spines[axis].set_linewidth(2)
    
plt.figure(4).tight_layout()
ax4 = plt.gca()
P35_MSDs.plot(x='time', y=P35_plot_array, kind='line', ax=ax4, xlim=(0,1.6), logy=True, ylim=(0.01,100), logx=True, color='purple', alpha=0.1, legend=False)
P35_MSDs.plot(x='time', y='average', kind='line', ax=ax4, xlim=(0,1.6), ylim=(0.01,100), logy=True, logx=True, linewidth=3.0, color='purple', alpha=1, legend=False)
ax4.set_ylabel('<MSD> (\u03BCm$^2$)', fontsize=18)
ax4.set_xlabel('lag time (s)', fontsize=18)
ax4.tick_params(labelsize=12)
for axis in ['top','bottom','left','right']:
    ax4.spines[axis].set_linewidth(2)

plt.figure(5).tight_layout()
ax5 = plt.gca()
P14_MSDs.plot(x='time', y=P14_plot_array, kind='line', ax=ax5, xlim=(0,1.6), ylim=(0.01,100), logy=True, logx=True, color='gray', alpha=0.05, legend=False)
P14_MSDs.plot(x='time', y='average', kind='line', ax=ax5, xlim=(0,1.6), ylim=(0.01,100), logy=True, logx=True, linewidth=4.0, color='gray', alpha=1, label='P14')
P21_MSDs.plot(x='time', y=P21_plot_array, kind='line', ax=ax5, xlim=(0,1.6), ylim=(0.01,100), logy=True, logx=True, color='goldenrod', alpha=0.05, legend=False)
P21_MSDs.plot(x='time', y='average', kind='line', ax=ax5, xlim=(0,1.6), ylim=(0.01,100), logy=True, logx=True, linewidth=4.0, color='goldenrod', alpha=1, label='P21')
P28_MSDs.plot(x='time', y=P28_plot_array, kind='line', ax=ax5, xlim=(0,1.6), logy=True, ylim=(0.01,100), logx=True, color='royalblue', alpha=0.05, legend=False)
P28_MSDs.plot(x='time', y='average', kind='line', ax=ax5, xlim=(0,1.6), ylim=(0.01,100), logy=True, logx=True, linewidth=4.0, color='royalblue', alpha=1, label='P28')
P35_MSDs.plot(x='time', y=P35_plot_array, kind='line', ax=ax5, xlim=(0,1.6), logy=True, ylim=(0.01,100), logx=True, color='purple', alpha=0.05, legend=False)
P35_MSDs.plot(x='time', y='average', kind='line', ax=ax5, xlim=(0,1.6), ylim=(0.01,100), logy=True, logx=True, linewidth=4.0, color='purple', alpha=1, label='P35')
ax5.set_ylabel('<MSD> (\u03BCm$^2$)', fontsize=18)
ax5.set_xlabel('lag time (s)', fontsize=18)
ax5.tick_params(labelsize=12)
for axis in ['top','bottom','left','right']:
    ax5.spines[axis].set_linewidth(2)

plt.show()